<a href="https://colab.research.google.com/github/GabrielDeml/trainingMobilenetV1SsdInColab/blob/master/wpilibCoralTrainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Prams

In [0]:
%tensorflow_version 1.x

Get classification transfer learning scripts.


In [0]:
%load_ext tensorboard

Go grab the models for the network and switch to the right version

In [0]:
!git clone https://github.com/tensorflow/models
%cd models/research/object_detection
!git checkout f788046ca876a8820e05b0b48c1fc2e16b0955bc

Set the paths

In [0]:
import os
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'
!CLASSIFY_DIR=/content/models/research/object_detection/

Grab the training scripts

In [0]:
!wget -O classify_scripts.tgz http://storage.googleapis.com/cloud-iot-edge-pretrained-models/docker/obj_det_scripts.tgz
!tar zxvf classify_scripts.tgz

Fix the scripts to get the training data

In [0]:
!sed -i '83s/object_detection\///' /content/models/research/object_detection/prepare_checkpoint_and_dataset.sh
!sed -i '86s/object_detection\///' /content/models/research/object_detection/prepare_checkpoint_and_dataset.sh
!cp /content/models/research/cognitive_planning/string_int_label_map_pb2.py /content/models/research/object_detection/protos

In [0]:
# mv /content/*.tar .
# tar -xf *.tar -C ./out/

Go get the training data

In [0]:
! ./prepare_checkpoint_and_dataset.sh --network_type mobilenet_v1_ssd

In [0]:
!rm learn_pet/pet/pet_faces*
!rm learn_pet/pet/pet_label*
!cp /content/map.pbtxt /content/models/research/object_detection/learn_pet/pet/
!cp /content/train.record /content/models/research/object_detection/learn_pet/pet

In [0]:
!sed -i '41s/object_detection\///' /content/models/research/object_detection/retrain_detection_model.sh

In [0]:
%cd /content/models/research/
!protoc --python_out=./ object_detection/protos/*.proto
%cd /content/models/research/object_detection/

In [0]:
%tensorboard --logdir=./learn_pet/train/

In [0]:
!./retrain_detection_model.sh

In [0]:
!sed -i '42s/object_detection\///' /content/models/research/object_detection/convert_checkpoint_to_edgetpu_tflite.sh

In [0]:
!./convert_checkpoint_to_edgetpu_tflite.sh --checkpoint_num 500

In [0]:
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
!echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list
!apt update
!apt install edgetpu

In [0]:
%cd /content/models/research/object_detection/learn_pet/models/
!edgetpu_compiler output_tflite_graph.tflite
%cd /content/models/research/object_detection/
!cp -r /content/models/research/object_detection/learn_pet/models/ /content/trained